# random seed average
https://gist.github.com/kunigaku/2e8efdd9822c934e3164d8dbc94a50bb
### model01-1でαとそれにに対するN数を決定し,このノートで提出データ作成

In [1]:
# import dependencies
import pandas as pd
import numerapi, os, datetime,time,gc
import sklearn.linear_model
from sklearn.linear_model import Lasso
from pycaret.regression import *

today = datetime.date.today()
dates = '2021-01-17'

In [ ]:
training_data = pd.read_csv('./csv/numerai_train_data.csv')#.set_index("id")
tournament_data = pd.read_csv('./csv/numerai_tournament_data.csv')#.set_index('id')
TARGET_NAME = [x for x in training_data.columns if 'target' in x][0]
PREDICTION_NAME = "prediction"
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
#training_features = training_data[feature_cols]

In [ ]:
def dataframe_cast_to_float32(df, features):
    typemap = {}
    for f in features:
        typemap[f] = 'float32'
    return df.astype(typemap)
# Submissions are scored by spearman correlation
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]

# convenience method for scoring
def score(df):
    return correlation(df[PREDICTION_NAME], df[TARGET_NAME])

training_data = dataframe_cast_to_float32(training_data, feature_cols)
tournament_data = dataframe_cast_to_float32(tournament_data, feature_cols)

In [ ]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "TIFYUYGPJCFQZT4SBLNYIOZWQWIMC2EQ"
secret_key = "QGLW6MHS7QIDPEELKVJ6DKUGBWHQMN7O3A2BCIV7CU4QKGTNVBZ7F5RRFF75M4DB"
model_id = "45b2b9e3-ed1b-4d82-bb2c-84f828b403fe"
model_2 = 'f624d92f-3965-4242-b069-dda40993fffa'
model_3 ='b58e9477-feb6-4cd3-ae53-290326a41ecd'
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

# n1

In [ ]:
lists = [0.000925] # 0.000925 n1
for i in lists:
    print(i)

In [ ]:
n_of_features =[]
for i in lists:
    lasso = Lasso(alpha=i).fit(training_data[feature_cols],training_data['target'])
    sum_features = np.sum(lasso.coef_ !=0)
    n_of_features.append([i,sum_features])
    #n_of_features.append(sum_features)

n_of_features

selected_features = []
for i,j in enumerate(lasso.coef_):
    if j !=0:
        selected_features.append(training_data.columns[i])

import pandas as pd
from xgboost import XGBRegressor

gc.collect()

predictions = []
for itr in range(30):
    model = XGBRegressor(max_depth=5, learning_rate=0.01,\
                         n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, \
                         seed=itr, tree_method='gpu_hist')
    model.fit(training_data[selected_features], training_data[TARGET_NAME])
    predictions.append(model.predict(tournament_data[selected_features]))

prediction_df = tournament_data["id"].to_frame()
prediction_df.fillna(0,inplace=True)
tournament_data[TARGET_NAME].fillna(0,inplace=True)
scores = []
mmc =[]
for prediction in predictions:
    scores.append(correlation(pd.Series(prediction), tournament_data[TARGET_NAME]))
sum(scores)/len(scores)

prediction_df = tournament_data["id"].to_frame()
for itr in range(len(predictions)):
    prediction_df[f"prediction_{itr}"] = predictions[itr]
pre_col_list = prediction_df.columns[prediction_df.columns.str.startswith('prediction_')]
pre_col_list
prediction_df[PREDICTION_NAME] = prediction_df[pre_col_list].mean(axis=1)
print('correlarion:',correlation(prediction_df[PREDICTION_NAME], tournament_data[TARGET_NAME]))

pred = pd.DataFrame()
pred["id"] = prediction_df['id']
pred["prediction"] = prediction_df['prediction']
pred.to_csv("predictions_xg_std_n1_seed_ave.csv", index=False)

submission_id = napi.upload_predictions(
    "predictions_xg_std_n1_seed_ave.csv", model_id= model_2 )

# n11

In [ ]:
 lists = [0.0007] 

In [ ]:
n_of_features =[]
for i in lists:
    lasso = Lasso(alpha=i).fit(training_data[feature_cols],training_data['target'])
    sum_features = np.sum(lasso.coef_ !=0)
    n_of_features.append([i,sum_features])
    #n_of_features.append(sum_features)

n_of_features

selected_features = []
for i,j in enumerate(lasso.coef_):
    if j !=0:
        selected_features.append(training_data.columns[i])

import pandas as pd
from xgboost import XGBRegressor

gc.collect()

predictions = []
for itr in range(30):
    model = XGBRegressor(max_depth=5, learning_rate=0.01,\
                         n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, \
                         seed=itr, tree_method='gpu_hist')
    model.fit(training_data[selected_features], training_data[TARGET_NAME])
    predictions.append(model.predict(tournament_data[selected_features]))

prediction_df = tournament_data["id"].to_frame()
prediction_df.fillna(0,inplace=True)
tournament_data[TARGET_NAME].fillna(0,inplace=True)
scores = []
mmc =[]
for prediction in predictions:
    scores.append(correlation(pd.Series(prediction), tournament_data[TARGET_NAME]))
sum(scores)/len(scores)

prediction_df = tournament_data["id"].to_frame()
for itr in range(len(predictions)):
    prediction_df[f"prediction_{itr}"] = predictions[itr]
pre_col_list = prediction_df.columns[prediction_df.columns.str.startswith('prediction_')]
pre_col_list
prediction_df[PREDICTION_NAME] = prediction_df[pre_col_list].mean(axis=1)
print('correlarion:',correlation(prediction_df[PREDICTION_NAME], tournament_data[TARGET_NAME]))

pred = pd.DataFrame()
pred["id"] = prediction_df['id']
pred["prediction"] = prediction_df['prediction']
pred.to_csv("predictions_xg_std_n11_seed_ave.csv", index=False)

submission_id = napi.upload_predictions(
    "predictions_xg_std_n11_seed_ave.csv", model_id= model_3 )

# n307

In [ ]:
%%time
import pandas as pd
from xgboost import XGBRegressor

gc.collect()

predictions = []
for itr in range(30):
    model = XGBRegressor(max_depth=5, learning_rate=0.01,\
                         n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, \
                         seed=itr, tree_method='gpu_hist')
    model.fit(training_data[feature_cols], training_data[TARGET_NAME])
    predictions.append(model.predict(tournament_data[feature_cols]))

tournament_data[TARGET_NAME].fillna(0,inplace=True)
scores = []
for prediction in predictions:
    scores.append(correlation(pd.Series(prediction), tournament_data[TARGET_NAME]))
sum(scores)/len(scores)

prediction_df = tournament_data["id"].to_frame()
for itr in range(len(predictions)):
    prediction_df[f"prediction_{itr}"] = predictions[itr]
prediction_df.fillna(0,inplace=True)
pre_col_list = prediction_df.columns[prediction_df.columns.str.startswith('prediction_')]
prediction_df[PREDICTION_NAME] = prediction_df[pre_col_list].mean(axis=1)
print('correlarion Model n307:',correlation(prediction_df[PREDICTION_NAME], tournament_data[TARGET_NAME]))

pred = pd.DataFrame()
pred["id"] = prediction_df['id']
pred["prediction"] = prediction_df['prediction']
pred.to_csv("predictions_xg_std_n311_seed_ave.csv", index=False)
submission_id = napi.upload_predictions(
    "predictions_xg_std_n311_seed_ave.csv", model_id= model_id )